In [1]:
from tqdm import tqdm

## 1. Pre-train on CIFAR-10

In [2]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cifar10.t7')
        best_acc = acc


for epoch in range(start_epoch, 350):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


  0%|          | 0/392 [00:00<?, ?it/s]


Epoch: 0


Acc: 47.020%:  99%|█████████▉| 100/101 [00:02<00:00, 44.70it/s]


Saving..


...

  0%|          | 0/392 [00:00<?, ?it/s]


Epoch: 348


  0%|          | 0/392 [00:00<?, ?it/s]


Epoch: 349


Acc: 96.544%: 100%|█████████▉| 391/392 [00:31<00:00, 12.95it/s]


testing accuracy: >90%

## Train on CINIC-10

- train: train set
- valid: valid set

In [5]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.0.t7')#'/tmp/work/checkpoint/ckpt.cifar10.t7')
net.load_state_dict(checkpoint['net'])
#start_epoch = checkpoint['epoch']
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 150, 300], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.0.t7')
        best_acc = acc


for epoch in range(start_epoch, 501):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..


  0%|          | 0/704 [00:00<?, ?it/s]

==> Building model..
==> Resuming from checkpoint..

Epoch: 0


  0%|          | 0/704 [00:00<?, ?it/s]

Saving..

Epoch: 1


...

  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 498


  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 499


Acc: 80.233%: 100%|██████████| 704/704 [01:25<00:00,  8.23it/s]


## Train on full CINIC-10

- train: train set + valid set
- valid: test set

In [2]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_train)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.0.t7')
net.load_state_dict(checkpoint['net'])
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 150, 300], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)+len(validloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)
        for batch_idx, (inputs, targets) in enumerate(validloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.1.t7')
        best_acc = acc

for epoch in range(start_epoch, 501):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
==> Building model..
==> Resuming from checkpoint..


  0%|          | 0/1408 [00:00<?, ?it/s]


Epoch: 0


  0%|          | 0/1408 [00:00<?, ?it/s]

Saving..

Epoch: 1


...


Epoch: 111


  0%|          | 0/1408 [00:00<?, ?it/s]


Epoch: 112


Acc: 78.255%:  76%|███████▌  | 1071/1408 [02:14<00:42,  7.84it/s]Process Process-475:
Process Process-476:
Acc: 78.255%:  76%|███████▌  | 1072/1408 [02:14<00:42,  7.94it/s]Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>


KeyboardInterrupt: 

## Train on full CINIC-10 with data augmentation

- train: train set + valid set
- valid: test set

In [ ]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *


from imgaug import augmenters as iaa
from imgaug import parameters as iap
import numpy as np
import PIL

class ImgAugTransform:
    def __init__(self):
        self.aug = iaa.Sequential([
            iaa.SomeOf((0, 3), [
                iaa.Affine(rotate=iap.Uniform(0.0, 360.0)),
                iaa.CropAndPad(percent=(-0.2, 0)),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.AdditiveGaussianNoise(scale=0.1*255),
                iaa.Sharpen(alpha=0.5),
                iaa.PiecewiseAffine(scale=iap.Uniform(0.02, 0.08), 
                                    nb_rows=iap.Uniform(4,8),
                                    nb_cols=iap.Uniform(4,8))
            ]),
        ])

    def __call__(self, img):
        img = np.array(img)
        return self.aug.augment_image(img)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    ImgAugTransform(),
    lambda x: PIL.Image.fromarray(x),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_train)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.1.t7')
net.load_state_dict(checkpoint['net'])
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 250, 450], gamma=0.5)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)+len(validloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)
        for batch_idx, (inputs, targets) in enumerate(validloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.2.t7')
        best_acc = acc


for epoch in range(start_epoch, 701):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
==> Building model..
==> Resuming from checkpoint..


  0%|          | 0/1408 [00:00<?, ?it/s]


Epoch: 0


Acc: 24.965%:  11%|█         | 154/1408 [02:48<24:46,  1.19s/it]

## Model Deployment

In [ ]:
_name = 'cinic10.0'
_path = '../mobileNetV2.{}.pth'.format(_name)

In [6]:
state = {
    'net': net.module.state_dict(),
    'acc': checkpoint['acc'],
    'epoch': checkpoint['epoch'],
}

torch.save(state, _path)

In [3]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load(_path)
net.load_state_dict(checkpoint['net'])

# Export model
print('==> Exporting model..')
dummy_input = Variable(torch.randn(1, 3, 32, 32), requires_grad=False)
torch.onnx.export(net, dummy_input, "../mobileNetV2.{}.onnx".format(_name), verbose=True)

==> Building model..
==> Resuming from checkpoint..
==> Exporting model..
graph(%0 : Float(1, 3, 32, 32)
      %1 : Float(32, 3, 3, 3)
      %2 : Float(32)
      %3 : Float(32)
      %4 : Float(32)
      %5 : Float(32)
      %6 : Float(32, 32, 1, 1)
      %7 : Float(32)
      %8 : Float(32)
      %9 : Float(32)
      %10 : Float(32)
      %11 : Float(32, 1, 3, 3)
      %12 : Float(32)
      %13 : Float(32)
      %14 : Float(32)
      %15 : Float(32)
      %16 : Float(16, 32, 1, 1)
      %17 : Float(16)
      %18 : Float(16)
      %19 : Float(16)
      %20 : Float(16)
      %21 : Float(16, 32, 1, 1)
      %22 : Float(16)
      %23 : Float(16)
      %24 : Float(16)
      %25 : Float(16)
      %26 : Float(96, 16, 1, 1)
      %27 : Float(96)
      %28 : Float(96)
      %29 : Float(96)
      %30 : Float(96)
      %31 : Float(96, 1, 3, 3)
      %32 : Float(96)
      %33 : Float(96)
      %34 : Float(96)
      %35 : Float(96)
      %36 : Float(24, 96, 1, 1)
      %37 : Float(24)
      %38 : F

RuntimeError: ONNX export failed: Couldn't export Python operator Scatter

Defined at:
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/parallel/scatter_gather.py(14): scatter_map
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/parallel/scatter_gather.py(16): scatter_map
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/parallel/scatter_gather.py(29): scatter
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/parallel/scatter_gather.py(36): scatter_kwargs
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py(121): scatter
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py(110): forward
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/modules/module.py(479): _slow_forward
/home/dllab/pytorch-cifar/env/lib/python3.6/site-packages/torch/nn/modules/module.py(489): __call__
/home/dllab/pytorch-cifar/env/lib/python3.6/site-pa

tensorRT deployment

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt
import numpy as np

def build_engine(onnx_model='./resnet18_food11.onnx', max_batch_size=256):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
        # Configure the builder here.
        builder.max_batch_size = max_batch_size
        builder.max_workspace_size = 1 <<  20
        # Parse the model to create a network.
        with open(onnx_model, 'rb') as model:
            parser.parse(model.read())
        # Build and return the engine. Note that the builder, network and parser are destroyed when this function returns.
        return builder.build_cuda_engine(network)
    
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
# Allocates all buffers required for an engine, i.e. host/device inputs/outputs.
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream

def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [ ]:
cuda.init()

max_batch_size = 1024
onnx_model = '../mobileNetV2.cifar10.onnx'
engine = build_engine(onnx_model, max_batch_size=max_batch_size)
context = engine.create_execution_context()

# input_image_paths = glob.glob('./data/*.npy')
# images = []
# labels = []
# for im_path in input_image_paths:
#     image = np.load(im_path)
#     images.append(image)
#     labels.append(os.path.basename(im_path).split(sep='_')[0])
# images = np.asarray(images)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])
testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

In [ ]:
batch_size = 64
print('batch size: ' + str(batch_size))

correct = 0
total = 0
inputs, outputs, bindings, stream = allocate_buffers(engine)
#for idx in range(0, len(images), batch_size):
for idx, (images, labels) in enumerate(testloader):
    images = np.asarray(images)
    inputs[0].host = images[idx: idx+batch_size]
    pred = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, 
                        stream=stream, batch_size=batch_size)
    pred = np.asarray(pred).reshape(-1,10)
    pred = np.argmax(pred)
    correct += np.sum(pred == labels)
    total += len(labels)
#     for i in range(len(images)):
#         predicted_label = str(np.argmax(pred[i]))
#         if(predicted_label == labels[idx+i]):
#             correct += 1
accuracy = correct / total
print('acc: ' + str(accuracy))

## Testing

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import sys
data_dir = "/tmp/work/data/CINIC-10/test/"
assert data_dir is not None, "No data directory"

from models import *
checkpoint = torch.load('../mobileNetV2.cinic10.0.pth')
model = MobileNetV2()
model.load_state_dict(checkpoint['net'])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def inference(model, testloader):
    total = 0
    correct = 0
    model.to(device)
    model = torch.nn.DataParallel(model)
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets.cuda()).sum().item()
    acc = 100.*correct/total
    print(acc)
    return acc

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
inference(model, testloader)